In [1]:
#https://github.com/C7C8/ds3001-critter-analysis/blob/master/scripts/decison-tree-month.py
from sklearn.ensemble import RandomForestClassifier
import json
import os
def dumpForest(forest, name, columns):
    if not os.path.isdir(name):
        os.makedirs(name)
#    #json part
#    for model_number, tree in enumerate(forest.estimators_):
#        fname = '{}/json/_{}.json'.format(name, model_number)
#        export_json(tree, fname, columns.tolist())
        
    #features
    featureCSVPath = '{}/features.csv'.format(name)
    if not os.path.isfile(featureCSVPath):
        open(featureCSVPath, 'a').close()
    with open(featureCSVPath, 'w') as f:
        f.write('feature\n')
        for feature in columns.tolist():
            f.write('{}\n'.format(feature))

In [ ]:
import os
import pandas as pd
#from https://www.kaggle.com/borismarjanovic/price-volume-data-for-all-us-stocks-etfs removing all stocks with less than 1 year of data
filePath = 'price-volume-data-for-all-us-stocks-etfs/Stocks'
fileList = os.listdir(filePath)
masterDF = pd.DataFrame({'Date': [],
                        'Open': [],
                        'High': [],
                        'Low': [],
                        'Close': [],
                        'Volume': [],
                        'VolumeZScore': [],
                        'highVsLowPerc': [],
                        'dayPercentChange': [],
                        'ticker': [],
                        'fiveDayAverage': [],
                        'tenDayAverage': [],
                        'fiveDayWeightedAverage': [],
                        'tenDayWeightedAverage': [],
                        'fiveDaySlopeChange': [],
                        'tenDaySlopeChange': [],
                        'fiveVsTenDaySlopeChange': [],
                        'fiveVsTenDayAverage': [],
                        'tmmrwChngAsPerc': [],
                        'zScoreOfChangeTmmrw': [],
                        'percentChangeInFiveDays': []},
                  index=[])   
counter = 0
for file in fileList:
    if counter % 50 == 0:
        print(counter)
    counter += 1 
    try:
        df = pd.read_csv(filePath + '/' + file, parse_dates = True, usecols = ['Date','Open','High','Low','Close','Volume'])
        print(df.head())
        tickerSymbol = file.split('.us.txt')[0]
        rowCount = df.shape[0]
        df = df.assign(VolumeZScore = [0.0]* rowCount, highVsLowPerc = [0.0]* rowCount, dayPercentChange = [0.0]* rowCount, ticker = [tickerSymbol]* df.shape[0], 
                       fiveDayAverage = [0.0]* rowCount, tenDayAverage = [0.0]* rowCount,
                       fiveDayWeightedAverage = [0.0]* rowCount, tenDayWeightedAverage = [0.0]* rowCount,
                       fiveDaySlopeChange = [0.0]* rowCount, tenDaySlopeChange = [0.0]* rowCount, fiveVsTenDaySlopeChange = [0.0]* rowCount,
                       fiveVsTenDayAverage = [0.0]* rowCount, tmmrwChngAsPerc = [0.0]* rowCount,zScoreOfChange = [0.0]*rowCount, 
                       percentChangeInFiveDays = [0.0]* rowCount)
        stdDevOfVolume = pd.Series.std(df['Volume'])
        meanOfVolume = pd.Series.mean(df['Volume'])
        df['VolumeZScore'] = (df['Volume'] - meanOfVolume).div(stdDevOfVolume)
        df['highVsLowPerc'] = ((df['High'] - df['Low']).div(df['Low']))
        df['dayPercentChange'] = ((df['Close'] - df['Open']).div(df['Open']))
        df['fiveDayAverage'] = ((df['Close'] + df['Close'].shift(1) + df['Close'].shift(2) + df['Close'].shift(3) + df['Close'].shift(4)).div(5))
        df['tenDayAverage'] = ((df['Close'] + df['Close'].shift(1) + df['Close'].shift(2) + df['Close'].shift(3) + df['Close'].shift(4) + 
                               (df['Close'].shift(5) + df['Close'].shift(6) + df['Close'].shift(7) + df['Close'].shift(8) + df['Close'].shift(9)).div(10)))
        df['fiveDayWeightedAverage'] = (((df['Close']*5) + (df['Close'].shift(1) * 4) + (df['Close'].shift(2) * 3) + 
                                         (df['Close'].shift(3) * 2) + (df['Close'].shift(4))).div(15))
        df['tenDayWeightedAverage'] = (((df['Close']*10) + (df['Close'].shift(1) * 9) + (df['Close'].shift(2) * 8) + 
                                         (df['Close'].shift(3) * 7) + (df['Close'].shift(4) * 6) + (df['Close'].shift(5) * 5)+
                                         (df['Close'].shift(7) * 3)+ (df['Close'].shift(8) * 2) + (df['Close'].shift(9))).div(55))
        df['fiveDaySlopeChange'] = ((df['Close']  - df['Open'].shift(4)).div(5))
        df['tenDaySlopeChange'] = ((df['Close']  - df['Open'].shift(9)).div(10))    
        df['fiveVsTenDaySlopeChange'] = (df['fiveDaySlopeChange'] - df['tenDaySlopeChange'])
        df['fiveVsTenDayAverage'] = (df['fiveDayAverage'] - df['tenDayAverage'])
        df['tmmrwChngAsPerc'] = ((df['Close'].shift(-1) - df['Open'].shift(-1)).div(df['Open'].shift(-1)))
        stdDevOfChangePercent = pd.Series.std(df['tmmrwChngAsPerc'])
        meanOfChangePercent = pd.Series.mean(df['tmmrwChngAsPerc'])
        df['zScoreOfChangeTmmrw'] = (df['tmmrwChngAsPerc'] - meanOfChangePercent).div(stdDevOfChangePercent)
        df['percentChangeInFiveDays'] = ((df['Close'].shift(-5) - df['Close']).div(df['Close']))
        df = df.dropna()
        masterDF = pd.concat([masterDF, df])
    except:
        print("error for: ", file)
print('finished cleaning')
masterDF.to_csv('masterDF.csv', index=False) 

0
         Date    Open    High     Low   Close    Volume
0  1999-11-18  30.713  33.754  27.002  29.702  66277506
1  1999-11-19  28.986  29.027  26.872  27.257  16142920
2  1999-11-22  27.886  29.702  27.044  29.702   6970266
3  1999-11-23  28.688  29.446  27.002  27.002   6332082
4  1999-11-24  27.083  28.309  27.002  27.717   5132147
         Date    Open    High     Low   Close  Volume
0  1970-01-02  2.2972  2.3178  2.2972  2.2972   26437
1  1970-01-05  2.2972  2.2972  2.2544  2.2759   29887
2  1970-01-06  2.2759  2.2759  2.2342  2.2342  106900
3  1970-01-07  2.2342  2.2342  2.2342  2.2342   85061
4  1970-01-08  2.2342  2.2342  2.2139  2.2139   66669
         Date  Open  High   Low  Close     Volume
0  1996-04-12  1.05  1.79  1.02   1.38  408720000
1  1996-04-15  1.49  1.50  1.25   1.34   79231200
2  1996-04-16  1.34  1.34  1.17   1.20   48026400
3  1996-04-17  1.18  1.18  1.03   1.12   42871200
4  1996-04-18  1.25  1.25  1.17   1.22   27352800
         Date   Open   High    Low  Cl

         Date    Open    High     Low   Close  Volume
0  2005-02-25  6.4987  6.6009  6.4668  6.5753   55766
1  2005-02-28  6.6072  6.7669  6.5944  6.6263   49343
2  2005-03-01  6.6391  6.6773  6.6072  6.6072   31643
3  2005-03-02  6.5753  6.6072  6.5434  6.5816   27101
4  2005-03-03  6.5753  6.6135  6.5562  6.5944   17387
         Date    Open    High     Low   Close    Volume
0  2007-11-12  2.5614  2.6144  2.4476  2.4492  36766891
1  2007-11-13  2.5113  2.6113  2.5113  2.5641  23295780
2  2007-11-14  2.6298  2.7217  2.6068  2.6497  45171831
3  2007-11-15  2.6208  2.6921  2.5635  2.5985  13383259
4  2007-11-16  2.6412  2.6593  2.5989  2.6419  21472065
         Date   Open   High    Low  Close  Volume
0  2005-02-25  15.70  15.75  15.24  15.35   92400
1  2005-02-28  15.35  15.55  15.13  15.20   98700
2  2005-03-01  15.21  15.86  15.15  15.47   97800
3  2005-03-02  15.65  15.77  15.25  15.31   61600
4  2005-03-03  15.55  16.00  15.52  15.80  138600
         Date    Open    High     Low   

         Date    Open    High     Low   Close  Volume
0  2005-02-25  7.6900  7.8800  7.6233  7.8200  636315
1  2005-02-28  7.7400  7.9600  7.6200  7.7933  771069
2  2005-03-01  7.8333  7.9567  7.7367  7.8900  955146
3  2005-03-02  7.8333  7.9500  7.7667  7.7933  703563
4  2005-03-03  7.8767  7.9267  7.6433  7.8367  676269
         Date   Open   High    Low  Close  Volume
0  2005-02-25  35.36  35.36  33.96  35.12  283990
1  2005-02-28  35.00  36.08  34.28  34.40  185515
2  2005-03-01  34.64  35.92  34.60  35.52  155093
3  2005-03-02  35.56  35.80  34.60  35.08  140085
4  2005-03-03  35.28  35.44  33.52  34.56  135703
         Date   Open   High    Low  Close    Volume
0  2007-05-09  20.00  20.00  20.00  20.00         0
1  2007-05-10  21.10  21.99  20.60  21.10  20008400
2  2007-05-11  21.13  22.40  21.13  21.85   3692300
3  2007-05-14  22.00  22.65  21.85  22.30   1652800
4  2007-05-15  21.85  22.42  20.73  21.00   2127300
         Date    Open    High     Low   Close   Volume
0  2005-0

         Date  Open  High   Low  Close   Volume
0  1985-07-01  0.49  0.51  0.49   0.49  6540000
1  1985-07-02  0.52  0.53  0.52   0.52  2505600
2  1985-07-03  0.52  0.53  0.52   0.52  1384800
3  1985-07-05  0.53  0.54  0.53   0.53  1221600
4  1985-07-08  0.52  0.52  0.51   0.51  1699200
         Date    Open    High     Low   Close   Volume
0  2005-02-25  16.551  16.701  16.391  16.683   428020
1  2005-02-28  16.657  16.979  16.596  16.657  1765159
2  2005-03-01  16.674  16.925  16.648  16.683   813696
3  2005-03-02  16.648  16.872  16.462  16.739   444005
4  2005-03-03  16.791  16.791  15.980  16.211  1021552
         Date   Open   High    Low  Close   Volume
0  2009-10-27  10.00  10.00  10.00  10.00        0
1  2009-10-28   9.40   9.50   8.37   8.49  2471423
2  2009-10-29   8.76   9.00   8.50   9.00   224137
3  2009-10-30   9.00   9.15   8.40   9.15   125686
4  2009-11-02   8.78   9.15   8.50   8.64    51683
         Date   Open   High    Low  Close   Volume
0  2014-07-31  25.00  28.

         Date    Open    High     Low   Close  Volume
0  2005-02-25  1920.0  1944.0  1890.0  1890.0      36
1  2005-02-28  1884.0  1950.0  1668.0  1890.0     111
2  2005-03-01  1824.0  1926.0  1824.0  1926.0      89
3  2005-03-02  1956.0  2016.0  1908.0  1920.0      26
4  2005-03-03  1920.0  1926.0  1842.0  1842.0      63
         Date    Open    High     Low   Close  Volume
0  2005-02-25  6.3089  6.3089  6.2453  6.2453    2424
1  2005-02-28  6.2453  6.4541  6.2181  6.4269   15643
2  2005-03-01  6.4450  6.4450  6.3089  6.3633    5464
3  2005-03-02  6.0819  6.4269  6.0819  6.4269    5850
4  2005-03-03  6.3724  6.3815  6.2726  6.2726    1763
         Date   Open   High    Low  Close  Volume
0  2005-02-25  9.715  9.787  9.701  9.781   44954
1  2005-02-28  9.737  9.787  9.694  9.730  123033
2  2005-03-01  9.701  9.768  9.684  9.722   93014
3  2005-03-02  9.662  9.690  9.642  9.669   48946
4  2005-03-03  9.627  9.701  9.627  9.701   60920
         Date    Open    High     Low   Close  Volum

...

         Date    Open    High     Low   Close  Volume
0  2005-02-25  5.9672  5.9852  5.9371  5.9793   58958
1  2005-02-28  5.9852  6.0215  5.9852  6.0093   26739
2  2005-03-01  6.0274  6.0576  5.9852  6.0515   39026
3  2005-03-02  6.0395  6.0515  6.0093  6.0515   31387
4  2005-03-03  6.0154  6.0395  5.9852  6.0215   25742
         Date    Open    High     Low   Close  Volume
0  2011-07-25  16.320  16.809  16.208  16.575  499034
1  2011-07-26  16.743  16.809  16.339  16.443  183115
2  2011-07-27  16.358  16.715  15.963  16.030  351696
3  2011-07-28  16.189  16.198  15.635  15.963  472977
4  2011-07-29  15.730  16.086  15.692  16.086  158399
         Date    Open    High     Low   Close    Volume
0  2013-01-18  10.744  10.902  10.179  10.321  26972793
1  2013-01-22  10.321  10.771  10.292  10.716   3212825
2  2013-01-23  10.750  10.818  10.633  10.811   1004121
3  2013-01-24  10.828  10.972  10.637  10.716   1811355
4  2013-01-25  10.744  10.953  10.581  10.851    783323
         Date   

         Date    Open    High     Low   Close  Volume
0  2005-02-25  15.145  15.621  14.948  15.539  195699
1  2005-02-28  15.211  15.539  14.593  14.628  225186
2  2005-03-01  14.619  14.684  13.657  14.076  309549
3  2005-03-02  13.952  14.693  13.930  14.158  219073
4  2005-03-03  14.058  14.512  14.058  14.346  480464
         Date    Open    High     Low   Close  Volume
0  2012-05-08  17.157  17.157  16.473  16.741   34544
1  2012-05-09  16.659  17.718  16.326  17.259   24225
2  2012-05-10  17.775  17.804  16.902  16.902   23090
3  2012-05-11  16.678  17.325  16.678  17.147   11454
4  2012-05-14  16.922  16.978  16.395  16.615   51458
         Date    Open    High     Low   Close  Volume
0  2005-02-25  1.9921  1.9921  1.8517  1.8517    1254
1  2005-02-28  1.8341  1.9569  1.7725  1.7725    3680
2  2005-03-01  1.9569  1.9569  1.7725  1.7725    1883
3  2005-03-02  1.8164  2.0008  1.7725  1.8955    2507
4  2005-03-03  1.8164  1.8867  1.7639  1.7639    1937
         Date    Open    Hig

         Date    Open    High     Low   Close  Volume
0  2013-09-27  17.531  17.580  17.439  17.439  128485
1  2013-09-30  17.484  17.625  17.484  17.625    9031
2  2013-10-01  17.625  17.717  17.625  17.670    2471
3  2013-10-02  17.670  17.756  17.670  17.717    5483
4  2013-10-03  17.670  17.858  17.670  17.812   22151
         Date    Open    High     Low   Close  Volume
0  2012-12-21  21.422  21.595  21.199  21.199   56585
1  2012-12-24  21.335  21.335  21.223  21.335    5955
2  2012-12-26  21.276  21.310  21.215  21.215   55478
3  2012-12-27  21.250  21.344  21.180  21.335   69491
4  2012-12-28  21.199  21.276  21.138  21.147   89646
         Date   Open   High    Low  Close  Volume
0  2005-02-25  15.44  15.44  15.44  15.44    1900
1  2005-02-28  15.45  15.61  15.45  15.61     800
2  2005-03-01  15.71  15.71  15.71  15.71     100
3  2005-03-03  15.71  15.72  15.71  15.72     200
4  2005-03-04  15.80  15.81  15.80  15.81     700
         Date    Open    High     Low   Close  Volum

         Date    Open    High     Low   Close  Volume
0  2005-02-25  15.645  15.930  15.636  15.645   19347
1  2005-02-28  15.890  15.981  15.645  15.890    7991
2  2005-03-01  15.914  16.201  15.890  15.914   68607
3  2005-03-02  15.055  15.527  15.055  15.055   54601
4  2005-03-03  15.249  15.283  15.132  15.249    9514
         Date    Open     High     Low   Close    Volume
0  2014-04-03  9.6670  10.5880  8.8723  8.8820  12241410
1  2014-04-04  8.9506   9.1957  8.5585  8.8624   1880964
2  2014-04-07  8.7349   8.7742  7.8330  7.9017   2613730
3  2014-04-08  7.9605   8.4345  7.9017  8.3331    978718
4  2014-04-09  8.3036   8.4899  8.0389  8.2742   1000665
         Date    Open    High     Low   Close   Volume
0  2005-02-25  12.462  12.555  12.435  12.555   673278
1  2005-02-28  12.348  12.421  12.234  12.300   478738
2  2005-03-01  12.381  12.416  12.340  12.381   344709
3  2005-03-02  12.194  12.300  12.174  12.262  1299938
4  2005-03-03  12.201  12.240  12.114  12.194  1063258
    

         Date    Open    High     Low   Close  Volume
0  2005-02-25  17.711  18.255  17.593  18.255  128722
1  2005-02-28  18.255  18.255  17.957  18.191  118097
2  2005-03-01  18.250  18.932  18.230  18.791  151401
3  2005-03-02  18.791  19.063  18.580  18.605  136894
4  2005-03-03  18.605  18.962  18.605  18.892   63341
         Date    Open    High     Low   Close  Volume
0  2012-05-15  9.1246  9.1246  9.1246  9.1246     268
1  2012-05-17  8.9836  8.9836  8.9836  8.9836     887
2  2012-05-22  9.0677  9.0677  9.0677  9.0677    1602
3  2012-05-23  9.1437  9.1437  8.9836  8.9836  165845
4  2012-05-24  8.9836  8.9836  8.9836  8.9836   31523
         Date    Open    High     Low   Close   Volume
0  1985-07-01  5.6267  5.6858  5.6267  5.6589   336180
1  1985-07-02  5.6589  5.6744  5.6267  5.6481   453306
2  1985-07-03  5.6481  5.6744  5.6060  5.6169   570444
3  1985-07-05  5.6169  5.6481  5.6060  5.6376   162764
4  1985-07-08  5.6376  5.7065  5.5694  5.7065  1232147
         Date    Open 

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import os
import pandas as pd
import pickle


masterDF = pd.read_csv('masterDF.csv', parse_dates = True)
zScoreAnswer = masterDF['zScoreOfChangeTmmrw']
zScoreAnswer = ['long' if x > .5 else 'short' if x < -.7 else 'hold' for x in zScoreAnswer ]
fiveDayChangeAnswer = masterDF['percentChangeInFiveDays']
fiveDayChangeAnswer = ['long' if x > .05 else 'short' if x < -.1 else 'hold' for x in fiveDayChangeAnswer]

masterDF = masterDF.drop(columns=['Date','zScoreOfChangeTmmrw','percentChangeInFiveDays', 'tmmrwChngAsPerc', \
                                  'ticker', 'High', 'Low', 'Open', 'Close', 'Volume'])
masterList = masterDF.values
masterTrainList, masterTestList, zScoreTrainList, zScoreTestList, fiveDayChangeTrainList, fiveDayChangeTestList =\
train_test_split(masterList,zScoreAnswer,fiveDayChangeAnswer,test_size = .25)


zScorePredictor = RandomForestClassifier(min_samples_split = 10, n_estimators = 50, random_state = 1)
zScorePredictor.fit(masterTrainList, zScoreTrainList)
fiveDayChangePredictor = RandomForestClassifier(min_samples_split = 10, n_estimators = 50, random_state = 1)
fiveDayChangePredictor.fit(masterTrainList, fiveDayChangeTrainList)
pickle.dump(zScorePredictor, open('zScorePredictor.pkl', "wb"))
pickle.dump(fiveDayChangePredictor, open('fiveDayChangePredictor.pkl', "wb"))
print('dumped')

zScorePredictions = zScorePredictor.predict(masterTestList)
zScorePredictionAccuracy = accuracy_score(zScoreTestList,zScorePredictions)
print('zScorePredictionAccuracy is : ', zScorePredictionAccuracy , '%')
fiveDayPrediction = fiveDayChangePredictor.predict(masterTestList)
fiveDayPredictionAccuracy = accuracy_score(fiveDayChangeTestList,fiveDayPrediction)
print('fiveDayPredictionAccuracy is : ', fiveDayPredictionAccuracy , '%')
dumpForest(zScorePredictor, 'zScorePredictor', masterDF.columns)
dumpForest(fiveDayChangePredictor, 'fiveDayChangePredictor', masterDF.columns)

zScoreFeature = pd.DataFrame(zScorePredictor.feature_importances_,
                                   index = masterDF.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)
print(zScoreFeature.head(10))
fiveDayFeature = pd.DataFrame(fiveDayChangePredictor.feature_importances_,
                                   index = masterDF.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)
print(fiveDayFeature.head(10))